# Bright Ceiling Line Detector
## Detecting lines from ceiling lights with intense bright centers

The challenge: Ceiling lights create bright lines, but the center is so intense it appears as a 'womb' (blob).
Solution: Use the bright center as a feature, not a bug!

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from scipy import ndimage

In [ ]:
def detect_bright_ceiling_lines(image, visualize=True):
    """
    Detect bright ceiling lines using the intense center as a guide.
    
    Strategy:
    1. Find super-bright regions (the 'womb' centers)
    2. Find moderately bright regions (the line extensions)
    3. Use skeleton/medial axis to find the line through the bright blob
    4. Extend the line using the surrounding bright regions
    """
    # Convert to grayscale if needed
    if len(image.shape) == 3:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray = image.copy()
    
    # Apply slight blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (5, 5), 1.0)
    
    # Step 1: Find SUPER bright regions (the intense centers)
    # These are the 'wombs' - use a very high threshold
    super_bright_threshold = np.percentile(blurred, 99.5)  # Top 0.5% brightest
    super_bright_mask = (blurred >= super_bright_threshold).astype(np.uint8) * 255
    
    # Step 2: Find moderately bright regions (the line extensions)
    # Use adaptive threshold based on image statistics
    bright_threshold = np.mean(blurred) + 2.0 * np.std(blurred)
    bright_mask = (blurred >= bright_threshold).astype(np.uint8) * 255
    
    # Step 3: Clean up the bright mask
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    bright_mask = cv2.morphologyEx(bright_mask, cv2.MORPH_CLOSE, kernel)
    bright_mask = cv2.morphologyEx(bright_mask, cv2.MORPH_OPEN, kernel)
    
    # Step 4: Find skeleton of bright regions to get centerlines
    # This helps us find the line through the 'womb'
    skeleton = cv2.ximgproc.thinning(bright_mask)
    
    # Step 5: Detect lines using Hough Transform on the skeleton
    lines = cv2.HoughLinesP(
        skeleton,
        rho=1,
        theta=np.pi/180,
        threshold=30,
        minLineLength=40,
        maxLineGap=30  # Allow larger gaps to bridge through the bright center
    )
    
    # Step 6: Filter lines that pass through or near super-bright regions
    # These are the lines we want!
    valid_lines = []
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            
            # Check if line passes through super-bright region
            # Sample points along the line
            num_samples = 20
            line_points = []
            for t in np.linspace(0, 1, num_samples):
                x = int(x1 + t * (x2 - x1))
                y = int(y1 + t * (y2 - y1))
                if 0 <= y < super_bright_mask.shape[0] and 0 <= x < super_bright_mask.shape[1]:
                    line_points.append(super_bright_mask[y, x])
            
            # If any point along the line is super-bright, keep it
            if line_points and max(line_points) > 0:
                # Calculate line length and angle
                length = np.sqrt((x2 - x1)**2 + (y2 - y1)**2)
                angle = np.abs(np.arctan2(y2 - y1, x2 - x1) * 180 / np.pi)
                
                valid_lines.append({
                    'coords': (x1, y1, x2, y2),
                    'length': length,
                    'angle': angle
                })
    
    # Visualization
    if visualize:
        fig, axes = plt.subplots(2, 3, figsize=(18, 12))
        
        # Original image
        axes[0, 0].imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB) if len(image.shape) == 3 else image, cmap='gray')
        axes[0, 0].set_title('Original Image')
        axes[0, 0].axis('off')
        
        # Super bright mask (the 'wombs')
        axes[0, 1].imshow(super_bright_mask, cmap='hot')
        axes[0, 1].set_title(f'Super Bright Centers (>{super_bright_threshold:.0f})')
        axes[0, 1].axis('off')
        
        # Bright mask (the lines)
        axes[0, 2].imshow(bright_mask, cmap='hot')
        axes[0, 2].set_title(f'Bright Regions (>{bright_threshold:.0f})')
        axes[0, 2].axis('off')
        
        # Skeleton
        axes[1, 0].imshow(skeleton, cmap='gray')
        axes[1, 0].set_title('Skeleton (Centerlines)')
        axes[1, 0].axis('off')
        
        # All detected lines
        result_all = image.copy()
        if lines is not None:
            for line in lines:
                x1, y1, x2, y2 = line[0]
                cv2.line(result_all, (x1, y1), (x2, y2), (0, 255, 0), 2)
        axes[1, 1].imshow(cv2.cvtColor(result_all, cv2.COLOR_BGR2RGB) if len(image.shape) == 3 else result_all)
        axes[1, 1].set_title(f'All Lines ({len(lines) if lines is not None else 0})')
        axes[1, 1].axis('off')
        
        # Valid lines (passing through bright centers)
        result_valid = image.copy()
        for line_info in valid_lines:
            x1, y1, x2, y2 = line_info['coords']
            cv2.line(result_valid, (x1, y1), (x2, y2), (0, 0, 255), 3)
            # Draw angle and length
            mid_x, mid_y = (x1 + x2) // 2, (y1 + y2) // 2
            cv2.putText(result_valid, f"{line_info['angle']:.0f}deg", 
                       (mid_x, mid_y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 1)
        axes[1, 2].imshow(cv2.cvtColor(result_valid, cv2.COLOR_BGR2RGB) if len(image.shape) == 3 else result_valid)
        axes[1, 2].set_title(f'Valid Ceiling Lines ({len(valid_lines)})')
        axes[1, 2].axis('off')
        
        plt.tight_layout()
        plt.show()
    
    return valid_lines, bright_mask, super_bright_mask, skeleton

In [ ]:
# Load the ceiling image
image = cv2.imread('data/ceiling.png')

if image is None:
    print("Error: Could not load image from data/ceiling.png")
else:
    print(f"Image loaded: {image.shape}")
    
    # Run the detector
    valid_lines, bright_mask, super_bright_mask, skeleton = detect_bright_ceiling_lines(image, visualize=True)
    
    print(f"\nDetected {len(valid_lines)} valid ceiling lines")
    for i, line_info in enumerate(valid_lines):
        print(f"Line {i+1}: Length={line_info['length']:.1f}px, Angle={line_info['angle']:.1f}°")

## Alternative Approach: Distance Transform Method

This method uses distance transform to find the ridge/centerline through bright blobs.

In [ ]:
def detect_lines_distance_transform(image, visualize=True):
    """
    Alternative method using distance transform to find centerlines.
    Works well when the bright 'womb' is elongated.
    """
    # Convert to grayscale
    if len(image.shape) == 3:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray = image.copy()
    
    # Blur
    blurred = cv2.GaussianBlur(gray, (5, 5), 1.0)
    
    # Threshold for bright regions
    threshold = np.mean(blurred) + 1.5 * np.std(blurred)
    _, bright_mask = cv2.threshold(blurred, threshold, 255, cv2.THRESH_BINARY)
    
    # Clean up
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7))
    bright_mask = cv2.morphologyEx(bright_mask, cv2.MORPH_CLOSE, kernel)
    
    # Distance transform
    dist_transform = cv2.distanceTransform(bright_mask, cv2.DIST_L2, 5)
    
    # Find ridge lines (local maxima in distance transform)
    # These represent the centerlines through the bright regions
    ridge_threshold = np.percentile(dist_transform[dist_transform > 0], 70)
    ridge_mask = (dist_transform >= ridge_threshold).astype(np.uint8) * 255
    
    # Thin to get skeleton
    skeleton = cv2.ximgproc.thinning(ridge_mask)
    
    # Detect lines
    lines = cv2.HoughLinesP(
        skeleton,
        rho=1,
        theta=np.pi/180,
        threshold=20,
        minLineLength=30,
        maxLineGap=25
    )
    
    if visualize:
        fig, axes = plt.subplots(2, 3, figsize=(18, 12))
        
        axes[0, 0].imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB) if len(image.shape) == 3 else image, cmap='gray')
        axes[0, 0].set_title('Original')
        axes[0, 0].axis('off')
        
        axes[0, 1].imshow(bright_mask, cmap='hot')
        axes[0, 1].set_title('Bright Mask')
        axes[0, 1].axis('off')
        
        axes[0, 2].imshow(dist_transform, cmap='jet')
        axes[0, 2].set_title('Distance Transform')
        axes[0, 2].axis('off')
        
        axes[1, 0].imshow(ridge_mask, cmap='gray')
        axes[1, 0].set_title('Ridge Lines')
        axes[1, 0].axis('off')
        
        axes[1, 1].imshow(skeleton, cmap='gray')
        axes[1, 1].set_title('Skeleton')
        axes[1, 1].axis('off')
        
        result = image.copy()
        if lines is not None:
            for line in lines:
                x1, y1, x2, y2 = line[0]
                cv2.line(result, (x1, y1), (x2, y2), (0, 0, 255), 3)
        axes[1, 2].imshow(cv2.cvtColor(result, cv2.COLOR_BGR2RGB) if len(image.shape) == 3 else result)
        axes[1, 2].set_title(f'Detected Lines ({len(lines) if lines is not None else 0})')
        axes[1, 2].axis('off')
        
        plt.tight_layout()
        plt.show()
    
    return lines

In [ ]:
# Test the distance transform method
if image is not None:
    print("\n=== Distance Transform Method ===")
    lines_dt = detect_lines_distance_transform(image, visualize=True)
    print(f"Detected {len(lines_dt) if lines_dt is not None else 0} lines")